In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import os
import cv2
import numpy as np
import tensorflow as tf
import scipy

from models import network, model_utils, inception_v1


os.sys.path.append("C:\\Users\\Tom\\Workspace\\KeywordSearch\\tensorflow")

In [ ]:
folder = "C:\\Users\\Tom\\Workspace\\KeywordSearch\\edges\\img"
filenames = [os.path.join(folder, f) for f in os.listdir(folder)]
selected_neuron = 888 #888 - computer; 696 - screen; 694 - keyboard; 692 - mouse; 960 - computer

@tf.RegisterGradient("GuidedRelu")
def _GuidedReluGrad(op, grad):
    gate_g = tf.cast(grad > 0, "float32")
    gate_y = tf.cast(op.outputs[0] > 0, "float32")
    return gate_y * gate_g * grad

In [ ]:
def guided_backprop(grad):
    image_2d = np.sum(np.abs(grad), axis=2)

    vmax = np.percentile(image_2d, 99)
    vmin = np.min(image_2d)

    return np.clip((image_2d - vmin) / (vmax - vmin), 0, 1)

def print_guided_backprop(filename, guided_backprop, probability, image):
    print(filename)
    
    top_prob = np.argsort(probability)[::-1]
    for i in range(len(top_prob)):
        if top_prob[i] == selected_neuron:
            break
    print(i, ":", probability[selected_neuron])
    
    
    fig = plt.figure(dpi=180)
    fig.add_subplot(131)
    plt.imshow(guided_backprop, cmap=plt.cm.gray, vmin=0, vmax=1)
    
    res = scipy.misc.imresize(guided_backprop*255, (10,10))
    fig.add_subplot(132)
    plt.imshow(res, cmap=plt.cm.gray)
    fig.add_subplot(133)
    plt.imshow(image, vmin=0, vmax=1)
    
    #plt.show()
    r = filename[:-3].decode('ASCII')
    print(r)
    plt.savefig(r + "guided_backprop.jpg")

In [ ]:
# mobile phone: 654
keys, images = network.get_image_as_batch(filenames, inception_v1.default_image_size, batch_size=1)

session = tf.Session()
session.run(tf.local_variables_initializer())

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=session, coord=coord)

neuron_selector = tf.placeholder(tf.int32)

with session.graph.gradient_override_map({'Relu': 'GuidedRelu'}):
    logits, end_points = network.build_net(images, 1390, scope='InceptionGeneralist', is_training=False)
    probabilities = tf.nn.softmax(logits, name='Probability')
    
    conv_layer = end_points["Mixed_5c"]
    target_neuron = logits[0][neuron_selector]
    grads = tf.gradients(target_neuron, images)[0]

session.run(tf.global_variables_initializer())
model_utils.restore_model(session, "C:\\Users\\Tom\\Workspace\\KeywordSearch\\tensorflow\\bin", None)



print(">> Classification started with neuron " + str(selected_neuron))

try:
    while not coord.should_stop():
        r_keys, r_prob, r_conv, r_grads, r_images = session.run([keys, probabilities, conv_layer, grads, images],
                                                                feed_dict={neuron_selector: selected_neuron})
        
        r_grad_cam = guided_backprop(r_grads[0])
        print_guided_backprop(r_keys[0], r_grad_cam, r_prob[0], r_images[0])
    
        for a in range(len(r_keys)):
            _, filename = os.path.split(r_keys[a])
            if isinstance(filenames, dict):
                file_id = filenames[r_keys[a].decode("utf-8")]
            else:
                file_id = int(filename[:-4])

            r_top_indexes = np.where(r_prob[a] >= 0.001)[0]
            m_len = len(r_top_indexes)
            r_top_vals = np.zeros(m_len)

            for b in range(m_len):
                r_top_vals[b] = r_prob[a][r_top_indexes[b]]
                
except tf.errors.OutOfRangeError:
    print(">> Classification completed.")
finally:
    # When done, ask the threads to stop.
    coord.request_stop()

    # Wait for threads to finish.
    coord.join(threads)
    session.close()
